# temporal - E07 - maximum-dissimilitude-analysis.ipynb
### Revised on: 2025 - 10 - 28

## Introduction

The **Maximum Dissimilarity Algorithm (MDA)** is a powerful technique used in environmental data analysis to select a representative subset of cases from a large dataset. This method is particularly useful when:

- Working with large time series that need to be reduced to manageable sizes
- Identifying representative environmental conditions for numerical modeling
- Creating synthetic scenarios that capture the full range of observed conditions

In this notebook, we demonstrate how to:
1. Apply MDA to select representative sea state cases from a long-term dataset
2. Use these cases to reconstruct data at a different location
3. Evaluate reconstruction performance using different interpolation methods

### Dataset
We use data from **SIMAR point 2041080** (Spanish coastal waters), which contains:
- `Hm0`: Significant wave height (m)
- `Tp`: Peak wave period (s)
- `DirM`: Mean wave direction (°)

## Step 1: Import Required Libraries

In [ ]:
# Standard libraries
import matplotlib.pyplot as plt
import pandas as pd

# environmentaltools modules
from environmentaltools.graphics import plots as fig
from environmentaltools.temporal.classification import mda, reconstruction
from environmentaltools.temporal import analysis
from environmentaltools.common import read, save

## Step 2: Configuration and Data Loading

Define the analysis parameters and load the source dataset.

In [ ]:
# Configuration parameters
ncasos = 500  # Number of representative cases to select
vars_ = ["Hm0", "Tp", "DirM"]  # Variables to analyze
mvar = "Hm0"  # Main variable for ordering cases
file_name = "SIMAR_2041080"  # Dataset identifier

# Load the data
data = read.PdE(f"data/{file_name}")

# Select only the variables of interest and remove noise
# nanoise() removes NaN values and applies noise reduction
data = analysis.nanoise(data, vars_)[vars_]

print(f"Dataset loaded: {len(data)} records")
print(f"Date range: {data.index[0]} to {data.index[-1]}")
print(f"\nData summary:")
print(data.describe())

## Step 3: Apply Maximum Dissimilarity Algorithm (MDA)

The MDA algorithm selects a subset of cases that maximizes the dissimilarity between them, ensuring that the selected cases span the entire range of observed conditions. This is particularly useful for:
- Reducing computational costs in numerical models
- Creating representative boundary conditions
- Capturing extreme and typical conditions

The algorithm works by:
1. Starting with the case having the maximum value of the main variable (`mvar`)
2. Iteratively selecting cases that are most dissimilar to already selected ones
3. Computing dissimilarity based on Euclidean distance in the normalized variable space

In [ ]:
# Apply MDA to select representative cases
fname = f"cases/cases_{ncasos}_{file_name}.csv"

# The mda function:
# - data: input dataset with all observations
# - vars_: list of variables to consider in dissimilarity calculation
# - ncasos: number of cases to select
# - mvar: main variable for initial case selection
# - fname: file path to save the selected cases
cases = mda(data, vars_, ncasos, mvar, fname)

print(f"\nMDA completed: {len(cases)} representative cases selected")
print(f"Cases saved to: {fname}")
print(f"\nSelected cases summary:")
print(cases.describe())

## Step 4: Visualize MDA Results

Create a visualization showing how the selected cases (in black) are distributed across the full dataset (in gray). This plot helps verify that:
- Selected cases cover the entire range of observations
- Extreme values are captured
- The distribution is well-represented

In [ ]:
# Create visualization of MDA selection
fname_fig = f"figures/mda_{ncasos}_{file_name}"

# plot_mda creates a 3D scatter plot (for 3 variables) showing:
# - All data points in gray with high transparency
# - Selected cases in black markers
fig.plot_mda(data, cases, vars_, fname=fname_fig)

print(f"MDA visualization saved to: {fname_fig}")

## Step 5: Load Transfer Cases

For data reconstruction, we need two sets of MDA cases:
- **Deep water cases**: From the original location (already computed above)
- **Shallow water cases**: From the target location where we want to reconstruct data

These paired cases establish the transfer function between locations.

In [ ]:
# Load MDA cases from deep water location (source)
cases_deep = read.csv(f"cases/cases_500_{file_name}.csv", index_col=True)

# Load corresponding cases from shallow water location (target)
# These cases represent the same environmental conditions at a different location
cases_shallow = read.csv("data/seastates_449100_4063000.csv", index_col=True)

print(f"Deep water cases: {len(cases_deep)} records")
print(f"Shallow water cases: {len(cases_shallow)} records")
print(f"\nDeep water summary:")
print(cases_deep.describe())
print(f"\nShallow water summary:")
print(cases_shallow.describe())

## Step 6: Load Time Series for Reconstruction

Load the full time series from the deep water location that needs to be reconstructed at the shallow water location.

In [ ]:
# Load simulation data from deep water (source location)
# This could be from climate models, hindcast data, or observations
file_name_sim = "data_CCLM4-CanESM2"
data_deep = pd.read_csv(f"data/{file_name_sim}.csv", index_col=0, parse_dates=True)

# Select variables and preprocess
data_deep = data_deep[["Hm0", "Tp", "DirM"]]
data_deep = analysis.nanoise(data_deep, ["Hm0", "Tp", "DirM"])[["Hm0", "Tp", "DirM"]]

print(f"Deep water time series: {len(data_deep)} records")
print(f"Period: {data_deep.index[0]} to {data_deep.index[-1]}")
print(f"\nStatistics:")
print(data_deep.describe())

## Step 7: Data Reconstruction Using Radial Basis Functions (RBF)

The reconstruction process uses the relationship established by the MDA cases to transfer data from one location to another. We test three interpolation methods:

1. **Linear**: Simple linear interpolation
2. **Nearest**: Nearest neighbor interpolation (fastest, less smooth)
3. **Cubic**: Cubic spline interpolation (smoothest, most computationally expensive)

### Reconstruction Process:
For each time step in the deep water data:
1. Find the closest MDA cases in the deep water set
2. Use the corresponding shallow water cases to interpolate the result
3. Apply the selected interpolation method (linear, nearest, or cubic)

### Parameters:
- `eps`: Shape parameter for RBF (1.0 is standard)
- `optimize`: Whether to optimize interpolation parameters
- `optimizer`: 'global' for global optimization
- `scale_data`: Whether to normalize data before interpolation

In [ ]:
# Define reconstruction parameters
index = data_deep.index  # Time index for reconstruction
base_vars = ["Hm0", "Tp", "DirM"]  # Variables to use as basis
recons_vars = ["Hm0", "Tp", "DirM"]  # Variables to reconstruct

# Test different interpolation methods
methods = ["linear", "nearest", "cubic"]

print("Starting reconstruction with different methods...")
print(f"Number of time steps to reconstruct: {len(index)}")
print(f"Number of MDA cases used: {ncasos}\n")

## Step 8: Perform Reconstruction and Visualize Results

For each interpolation method, we:
1. Reconstruct the data at the target location
2. Create comparison plots showing:
   - Original (deep) vs reconstructed (shallow) data
   - MDA cases used for the transfer function
3. Save the reconstructed data to CSV files

In [ ]:
# Loop through interpolation methods
for method in methods:
    print(f"\n{'='*60}")
    print(f"Processing method: {method.upper()}")
    print(f"{'='*60}")
    
    # Perform reconstruction
    # The reconstruction function uses RBF interpolation to transfer data
    # from deep water (cases_deep) to shallow water (cases_shallow)
    data_reconstructed = reconstruction(
        cases_deep,        # MDA cases from source location
        data_deep,         # Full time series to reconstruct
        cases_shallow,     # Corresponding MDA cases at target location
        index,             # Time index for output
        base_vars,         # Variables to use as input
        recons_vars,       # Variables to reconstruct
        method=method,     # Interpolation method
        eps=1.0,           # RBF shape parameter
        optimize=True,     # Optimize interpolation parameters
        optimizer="global", # Use global optimization
        num=ncasos,        # Number of cases to use
        scale_data=False,  # Don't normalize data
        scaler_method="MinMaxScaler",  # Scaler type (if scale_data=True)
    )
    
    print(f"Reconstruction completed: {len(data_reconstructed)} records")
    print(f"Reconstructed data summary:")
    print(data_reconstructed.describe())
    # Create comparison plots
    # Three subplots, one for each variable (Hm0, Tp, DirM)
    _, axs = plt.subplots(1, 3, figsize=(12, 5))
    axs = axs.flatten()
    
    # Plot 1: Significant Wave Height (Hm0)
    axs[0].plot(
        data_deep["Hm0"],              # Deep water data (x-axis)
        data_reconstructed["Hm0"],     # Reconstructed shallow water (y-axis)
        ".",                           # Point markers
        color="cyan",                  # Cyan color for reconstructed data
        alpha=0.5,                     # Semi-transparent
        label="RBF reconstruction"
    )
    axs[0].plot(
        cases_deep["Hm0"],             # MDA cases deep water
        cases_shallow["Hm0"],          # MDA cases shallow water
        "+k",                          # Black crosses
        markersize=8,
        label="MDA selection"
    )
    axs[0].plot([0, 10], [0, 10], "k--", alpha=0.3, label="1:1 line")  # Perfect fit reference
    axs[0].set_xlabel(r"Deep water $H_{m0}$ (m)")
    axs[0].set_ylabel(r"Shallow water $H_{m0}$ (m)")
    axs[0].legend()
    axs[0].grid(True, alpha=0.3)
    
    # Plot 2: Peak Period (Tp)
    axs[1].plot(
        data_deep["Tp"], 
        data_reconstructed["Tp"], 
        ".", 
        color="cyan",
        alpha=0.5
    )
    axs[1].plot(cases_deep["Tp"], cases_shallow["Tp"], "+k", markersize=8)
    axs[1].plot([0, 25], [0, 25], "k--", alpha=0.3)
    axs[1].set_xlabel(r"Deep water $T_p$ (s)")
    axs[1].set_ylabel(r"Shallow water $T_p$ (s)")
    axs[1].grid(True, alpha=0.3)
    
    # Plot 3: Mean Direction (DirM)
    axs[2].plot(
        data_deep["DirM"], 
        data_reconstructed["DirM"], 
        ".", 
        color="cyan",
        alpha=0.5
    )
    axs[2].plot(cases_deep["DirM"], cases_shallow["DirM"], "+k", markersize=8)
    axs[2].plot([0, 360], [0, 360], "k--", alpha=0.3)
    axs[2].set_xlabel(r"Deep water $\theta_m$ ($^\circ$)")
    axs[2].set_ylabel(r"Shallow water $\theta_m$ ($^\circ$)")
    axs[2].grid(True, alpha=0.3)
    
    # Overall plot formatting
    plt.suptitle(f"Reconstruction using {method.upper()} interpolation", fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    # Save figure
    fig_path = f"figures/reconstruction_{method}.png"
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    print(f"Figure saved: {fig_path}")
    plt.show()

    # Save reconstructed data to CSV
    output_file = f"data/SIMAR_2041080_reconstructed_449100_4063000_{method}.csv"
    save.to_csv(data_reconstructed, output_file)
    print(f"Reconstructed data saved: {output_file}\n")

## Results Interpretation

### What to look for in the plots:

1. **Scatter around the 1:1 line**: Points should cluster around the diagonal (black dashed line), indicating good reconstruction

2. **MDA cases (black crosses)**: These define the transfer function. They should:
   - Cover the range of observed values
   - Show a clear trend/relationship between locations

3. **Cyan points (reconstructed data)**: 
   - Tighter clustering = better reconstruction
   - Spread indicates uncertainty in the method

### Comparison of methods:

- **Linear**: Good balance between accuracy and speed. Recommended for most applications.
- **Nearest**: Fastest but less smooth. May show step-like behavior.
- **Cubic**: Smoothest results but can oscillate beyond MDA case ranges.

### Typical applications:

- Downscaling climate model data to coastal locations
- Wave propagation from offshore to nearshore
- Transfer of data between measurement locations
- Creating boundary conditions for high-resolution models

## Summary

This notebook demonstrated the **Maximum Dissimilarity Algorithm (MDA)** workflow for environmental data analysis:

### Key steps completed:
1. ✅ Loaded and preprocessed sea state data
2. ✅ Applied MDA to select 500 representative cases
3. ✅ Visualized the distribution of selected cases
4. ✅ Loaded transfer functions between deep and shallow water
5. ✅ Reconstructed data using three interpolation methods (linear, nearest, cubic)
6. ✅ Created comparison plots to evaluate reconstruction quality
7. ✅ Saved reconstructed datasets for further analysis

### Output files:
- **MDA cases**: `cases/cases_500_SIMAR_2041080.csv`
- **Reconstructed data**: `data/SIMAR_2041080_reconstructed_449100_4063000_[method].csv`
- **Figures**: `figures/mda_500_SIMAR_2041080.png` and `figures/reconstruction_[method].png`

### Next steps:
- Validate reconstructed data against observations (if available)
- Calculate error metrics (RMSE, bias, correlation)
- Use reconstructed data for coastal engineering applications
- Experiment with different numbers of MDA cases (ncasos)

---

**References:**
- Camus, P., et al. (2011). A hybrid efficient method to downscale wave climate to coastal areas. *Coastal Engineering*, 58(9), 851-862.
- Cobos, M., et al. (2020). A model to study the consequences of human actions in the Guadalquivir River Estuary. *PhD Thesis, University of Granada*.